In [ ]:
import os, math, time, random
import numpy as np
from tqdm import tqdm
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision, torchvision.transforms as T
import matplotlib.pyplot as plt

# ----------------- Utility -----------------
def count_params(m):
    total = sum(p.numel() for p in m.parameters())
    return total, sum(p.numel() for p in m.parameters() if p.requires_grad)

def try_flops(model, input_size=(1,3,32,32)):
    try:
        from thop import profile
        x = torch.randn(*input_size).to(next(model.parameters()).device)
        macs,_ = profile(model, inputs=(x,), verbose=False)
        return 2*macs
    except:
        return None

# ----------------- DropPath -----------------
class DropPath(nn.Module):
    def __init__(self, p=0.): super().__init__(); self.p=p
    def forward(self,x):
        if not self.training or self.p==0.: return x
        keep = 1-self.p
        shape = (x.shape[0],)+(1,)*(x.ndim-1)
        rand = x.new_empty(shape).bernoulli_(keep)
        return x/keep*rand

# ----------------- Building blocks -----------------
class DWConv(nn.Module):
    def __init__(self,dim): super().__init__()
    def forward(self,x,H,W):
        B,N,C=x.shape
        x=x.transpose(1,2).reshape(B,C,H,W)
        x=F.conv2d(x,x.new_ones((C,1,3,3))/9,groups=C,padding=1)
        return x.reshape(B,C,-1).transpose(1,2)

class FFN(nn.Module):
    def __init__(self,dim,hidden,drop):
        super().__init__()
        self.net=nn.Sequential(nn.Linear(dim,hidden),nn.GELU(),nn.Dropout(drop),
                               nn.Linear(hidden,dim),nn.Dropout(drop))
    def forward(self,x): return self.net(x)

class CascadedGroupAttention(nn.Module):
    def __init__(self, dim, h=4, qk_ratio=0.25):
        super().__init__()
        self.h = h
        self.head_dim = dim // h
        self.qk_dim = int(self.head_dim * qk_ratio)
        self.qp = nn.ModuleList([nn.Linear(self.head_dim, self.qk_dim, bias=False) for _ in range(h)])
        self.kp = nn.ModuleList([nn.Linear(self.head_dim, self.qk_dim, bias=False) for _ in range(h)])
        self.vp = nn.ModuleList([nn.Linear(self.head_dim, self.head_dim, bias=False) for _ in range(h)])
        self.out = nn.Linear(dim, dim)

    def forward(self, x, H, W):
        B, N, C = x.shape
        xs = x.reshape(B, N, self.h, self.head_dim).permute(2, 0, 1, 3)
        outs = []
        prev = None
        for j in range(self.h):
            xj = xs[j]
            if prev is not None:
                xj = xj + prev   
            q, k, v = self.qp[j](xj), self.kp[j](xj), self.vp[j](xj)
            att = (q @ k.transpose(-2, -1)) / math.sqrt(self.qk_dim)
            att = att.softmax(-1)
            out = att @ v
            outs.append(out)
            prev = out.detach()  
        out = torch.cat(outs, dim=-1)
        return self.out(out)

class Block(nn.Module):
    def __init__(self,dim,h,mlp_ratio,drop_path,qk_ratio):
        super().__init__()
        hidden=int(dim*mlp_ratio)
        self.norm1,self.ffn1=nn.LayerNorm(dim),FFN(dim,hidden,0.)
        self.norm2,self.dw1=nn.LayerNorm(dim),DWConv(dim)
        self.norm3,self.attn=nn.LayerNorm(dim),CascadedGroupAttention(dim,h,qk_ratio)
        self.norm4,self.ffn2=nn.LayerNorm(dim),FFN(dim,hidden,0.)
        self.dp=DropPath(drop_path)
    def forward(self,x,H,W):
        x=x+self.dp(self.ffn1(self.norm1(x)))
        x=x+self.dp(self.dw1(self.norm2(x),H,W))
        x=x+self.dp(self.attn(self.norm3(x),H,W))
        x=x+self.dp(self.ffn2(self.norm4(x)))
        return x

# ----------------- EfficientViT-Tiny -----------------
class EfficientViT_Tiny(nn.Module):
    def __init__(self,num_classes=10,stage_c=(64,128,192),stage_d=(1,2,3),stage_h=(2,4,4),
                 mlp_ratio=2.,qk_ratio=0.25,dp_rate=0.1):
        super().__init__()
        self.stem=nn.Sequential(
            nn.Conv2d(3,stage_c[0],3,1,1,bias=False),nn.BatchNorm2d(stage_c[0]),nn.ReLU(True),
            nn.Conv2d(stage_c[0],stage_c[0],3,2,1,bias=False),nn.BatchNorm2d(stage_c[0]),nn.ReLU(True))
        self.stages=nn.ModuleList(); self.proj=nn.ModuleList()
        total=sum(stage_d); dp_rates=torch.linspace(0,dp_rate,total).tolist(); idx=0
        for i,(C,L,H) in enumerate(zip(stage_c,stage_d,stage_h)):
            self.proj.append(nn.Sequential(nn.Conv2d(C,C,1,bias=False),nn.BatchNorm2d(C),nn.ReLU(True)))
            blks=[]
            for _ in range(L):
                blks.append(Block(C,H,mlp_ratio,dp_rates[idx],qk_ratio)); idx+=1
            self.stages.append(nn.ModuleList(blks))
            if i<len(stage_c)-1:
                setattr(self,f"down{i}",nn.Sequential(
                    nn.Conv2d(C,stage_c[i+1],3,2,1,bias=False),
                    nn.BatchNorm2d(stage_c[i+1]),nn.ReLU(True)))
        self.pool=nn.AdaptiveAvgPool2d(1)
        self.head=nn.Linear(stage_c[-1],num_classes)
    def forward(self,x):
        x=self.stem(x)
        for i,blks in enumerate(self.stages):
            B,C,H,W=x.shape
            for blk in blks:
                t=x.flatten(2).transpose(1,2)
                t=blk(t,H,W)
                x=t.transpose(1,2).reshape(B,C,H,W)
            if i<len(self.stages)-1:
                x=getattr(self,f"down{i}")(x)
                x=self.proj[i+1](x)
        return self.head(self.pool(x).flatten(1))

# ----------------- CIFAR-10  -----------------
def get_cifar10(bs=128):
    aug=T.Compose([
        T.RandAugment(num_ops=2,magnitude=9),
        T.RandomCrop(32,padding=4),T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize((0.4914,0.4822,0.4465),(0.2470,0.2435,0.2616))])
    norm=T.Compose([
        T.ToTensor(),
        T.Normalize((0.4914,0.4822,0.4465),(0.2470,0.2435,0.2616))])
    tr=torchvision.datasets.CIFAR10('./data',train=True,download=True,transform=aug)
    te=torchvision.datasets.CIFAR10('./data',train=False,download=True,transform=norm)
    return DataLoader(tr,batch_size=bs,shuffle=True,num_workers=4),DataLoader(te,batch_size=bs,shuffle=False,num_workers=4)

# ----------------- Mixup / CutMix -----------------
def cutmix_box(size,lam):
    W,H=size[3],size[2]
    cut_rat=np.sqrt(1-lam)
    cut_w,cut_h=int(W*cut_rat),int(H*cut_rat)
    cx,cy=np.random.randint(W),np.random.randint(H)
    bbx1=np.clip(cx-cut_w//2,0,W); bby1=np.clip(cy-cut_h//2,0,H)
    bbx2=np.clip(cx+cut_w//2,0,W); bby2=np.clip(cy+cut_h//2,0,H)
    return bbx1,bby1,bbx2,bby2

def mixup_cutmix(x,y,alpha=1.0,cutmix_prob=0.5):
    lam=np.random.beta(alpha,alpha)
    rand_index=torch.randperm(x.size(0)).to(x.device)
    if np.random.rand()<cutmix_prob:
        bbx1,bby1,bbx2,bby2=cutmix_box(x.size(),lam)
        x[:,:,bby1:bby2,bbx1:bbx2]=x[rand_index,:,bby1:bby2,bbx1:bbx2]
        lam=1-((bbx2-bbx1)*(bby2-bby1)/(x.size(-1)*x.size(-2)))
    y_a,y_b=y,y[rand_index]
    return x,y_a,y_b,lam

# ----------------- Train/Eval -----------------
def train_one_epoch(m,dl,opt,device,loss_fn,alpha=1.0):
    m.train(); total_loss=0; c1=c5=n=0
    for x,y in tqdm(dl,leave=False):
        x,y=x.to(device),y.to(device)
        x,y_a,y_b,lam=mixup_cutmix(x,y,alpha)
        opt.zero_grad()
        out=m(x)
        loss=lam*loss_fn(out,y_a)+(1-lam)*loss_fn(out,y_b)
        loss.backward(); opt.step()
        total_loss+=loss.item()*x.size(0); n+=x.size(0)
        p=out.topk(5,1)[1]
        c1+=(lam*(p[:,0]==y_a).sum()+(1-lam)*(p[:,0]==y_b).sum()).item()
        c5+=sum([lam*(y_a[i].item() in p[i].tolist())+(1-lam)*(y_b[i].item() in p[i].tolist()) for i in range(y.size(0))])
    return total_loss/n,c1/n,c5/n

def evaluate(m,dl,device,loss_fn):
    m.eval(); L=c1=c5=n=0
    with torch.no_grad():
        for x,y in dl:
            x,y=x.to(device),y.to(device)
            o=m(x); loss=loss_fn(o,y)
            L+=loss.item()*x.size(0); n+=x.size(0)
            p=o.topk(5,1)[1]
            c1+=(p[:,0]==y).sum().item()
            c5+=sum([1 if y[i].item() in p[i].tolist() else 0 for i in range(y.size(0))])
    return L/n,c1/n,c5/n

# ----------------- Plot -----------------
def plot_hist(hist,out='./outputs'):
    os.makedirs(out,exist_ok=True)
    e=range(1,len(hist['train_loss'])+1)
    plt.figure(); plt.plot(e,hist['train_loss'],label='train');plt.plot(e,hist['val_loss'],label='val')
    plt.xlabel('Epoch');plt.ylabel('Loss');plt.legend();plt.grid();plt.title('Loss');plt.savefig(f'{out}/loss.png');plt.close()
    plt.figure();plt.plot(e,np.array(hist['train_top1'])*100,label='train');plt.plot(e,np.array(hist['val_top1'])*100,label='val')
    plt.plot(e,np.array(hist['val_top5'])*100,label='val_top5');plt.xlabel('Epoch');plt.ylabel('Accuracy %')
    plt.legend();plt.grid();plt.title('Accuracy');plt.savefig(f'{out}/accuracy.png');plt.close()

# ----------------- Main -----------------
def main():
    torch.autograd.set_detect_anomaly(False)
    device='cuda' if torch.cuda.is_available() else 'cpu'
    batch,epochs,lr,wd=128,150,3e-4,0.05
    patience = 15  
    wait = 0

    model=EfficientViT_Tiny(
        num_classes=10,
        stage_c=(64,128,192),
        stage_d=(1,2,3),
        stage_h=(2,4,4),
        mlp_ratio=2.0,
        qk_ratio=0.25,
        dp_rate=0.1
    ).to(device)

    p,_=count_params(model); print(f'Params: {p/1e6:.3f}M')
    fl=try_flops(model); 
    if fl: print(f'FLOPs ≈ {fl/1e6:.1f}M')
    tr,te=get_cifar10(batch)
    opt=torch.optim.AdamW(model.parameters(),lr=lr,weight_decay=wd)
    sch=torch.optim.lr_scheduler.CosineAnnealingLR(opt,T_max=epochs)
    loss_fn=nn.CrossEntropyLoss(label_smoothing=0.1)
    hist={'train_loss':[],'train_top1':[],'train_top5':[],
          'val_loss':[],'val_top1':[],'val_top5':[]}
    best=0; os.makedirs('outputs',exist_ok=True)
    for ep in range(1,epochs+1):
        tl,t1,t5=train_one_epoch(model,tr,opt,device,loss_fn)
        vl,v1,v5=evaluate(model,te,device,loss_fn)
        sch.step()
        hist['train_loss'].append(tl);hist['train_top1'].append(t1);hist['train_top5'].append(t5)
        hist['val_loss'].append(vl);hist['val_top1'].append(v1);hist['val_top5'].append(v5)
        if v1>best:
            best=v1; wait=0
            torch.save(model.state_dict(),'outputs/best_efficientvit_tiny_cifar10.pth')
        else:
            wait += 1
        print(f'E{ep:03d}/{epochs} | TL {tl:.3f} | VL {vl:.3f} | T1 {t1*100:.2f}% | V1 {v1*100:.2f}% | V5 {v5*100:.2f}% | Wait {wait}')
        if wait >= patience:
            print(f'⏹ Early stopping at epoch {ep} (no val acc improvement for {patience} epochs)')
            break
    plot_hist(hist)
    print(f'Best val Top-1: {best*100:.2f}%')

if __name__=='__main__':
    main()


Params: 1.792M


100%|██████████| 170M/170M [00:02<00:00, 76.4MB/s]


E001/150 | TL 2.213 | VL 1.741 | T1 21.94% | V1 43.21% | V5 90.95% | Wait 0


E002/150 | TL 2.112 | VL 1.602 | T1 28.51% | V1 53.34% | V5 94.88% | Wait 0


E003/150 | TL 2.013 | VL 1.593 | T1 35.14% | V1 55.97% | V5 94.51% | Wait 0


...

E146/150 | TL 1.584 | VL 0.756 | T1 57.94% | V1 91.49% | V5 99.69% | Wait 7


E147/150 | TL 1.587 | VL 0.815 | T1 57.67% | V1 91.47% | V5 99.63% | Wait 8


E148/150 | TL 1.606 | VL 0.745 | T1 57.54% | V1 91.41% | V5 99.72% | Wait 9


E149/150 | TL 1.593 | VL 0.805 | T1 57.17% | V1 91.51% | V5 99.63% | Wait 10


E150/150 | TL 1.644 | VL 0.791 | T1 55.32% | V1 91.53% | V5 99.66% | Wait 11
Best val Top-1: 91.54%
